In [ ]:
!pip install scipy==1.10.1 scikit-image==0.19.3 vit_keras==0.1.2
!pip install memory_profiler

In [ ]:
import sys
import io
import gc
import ctypes
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import cv2
import os
import cv2
import random
import shutil
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.client import device_lib
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import numpy as np
from vit_keras import vit, utils
import lime
import skimage
import shap
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_image import LimeImageExplainer
import ctypes

def preprocess_for_attack(image):

    # 이미지 정규화
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std

    return image

def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR) #크기 조절
    image = tf.image.central_crop(image, central_fraction=crop_size / resize_size) #중앙 224x224
    image = tf.math.divide(image, 255.0) #normalize
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std #다 normalize

    return image


def resize_and_crop(image, resize_size=256, crop_size=224):
    # Resize with bilinear interpolation
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR)
    cropped_image = tf.image.central_crop(resized_image, central_fraction=crop_size / resize_size)
    image = tf.math.divide(cropped_image, 255.0) #normalize

    return image

import copy
from skimage import color

def deepfool(model, image, num_classes=2, overshoot=0.02, max_iter=50, shape=(224, 224, 3)):
    #image = preprocess_image(image)
    #image_array = np.array(image)
    #print(np.shape(image_array)) # 28*28

    
    #image_norm = tf.cast(image_array - 0.5, tf.float32)
    #image_norm = np.reshape(image_norm, shape)  # 28*28*1
    #image_norm = image_norm[tf.newaxis, ...]  # 1*28*28*1
    
    image_norm = preprocess_for_attack(image)

    f_image = model(image_norm).numpy().flatten()
    I = (np.array(f_image)).flatten().argsort()[::-1]
    I = I[0:num_classes]
    label = I[0]
    # print(label, "label")

    input_shape = np.shape(image)
    pert_image = copy.deepcopy(image)
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)

    loop_i = 0
    x = tf.Variable(pert_image)
    #fs = model(preprocess_for_attack(x))
    k_i = label

    #print(fs)  # shape=(1, 10)

    def loss_func(logits, I, k):
        # return tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
        return logits[0, I[k]]

    while k_i == label and loop_i < max_iter:

        pert = np.inf

        for k in range(0, num_classes):
            one_hot_label_k = tf.one_hot(I[k], num_classes)
            with tf.GradientTape() as tape:
                tape.watch(x)
                fs = model(preprocess_for_attack(x))
                loss_value = loss_func(fs, I, k)
            cur_grad = tape.gradient(loss_value, x)

            w_k = cur_grad

            f_k = (fs[0, I[k]]).numpy()

            pert_k = abs(f_k) / np.linalg.norm(tf.reshape(w_k, [-1]))

            if pert_k < pert:
                pert = pert_k
                w = w_k

        r_i = (pert + 1e-4) * w / np.linalg.norm(w)
        r_tot = np.float32(r_tot + r_i)

        pert_image = image + (1 + overshoot) * r_tot

        x = tf.Variable(pert_image)

        fs = model(preprocess_for_attack(x))
        k_i = np.argmax(np.array(fs).flatten())

        loop_i += 1

    r_tot = (1 + overshoot) * r_tot
    
    #pert_image = np.reshape(pert_image, (224, 224, 3))

    return pert_image



def deepfool_attack_transparency(model, masked_image, original_image, overshoot = 0.001, max_iter = 30, num_classes = 2):
    masked_image = np.expand_dims(masked_image, axis=0)
    perturbed_image = tf.cast(masked_image, tf.float32)
    
    image_norm = preprocess_for_attack(original_image.numpy().reshape(1, 224, 224, 3))

    f_image = model(image_norm).numpy().flatten()
    I = (np.array(f_image)).flatten().argsort()[::-1]
    I = I[0:num_classes]
    label = I[0]
    #print(label, "label")

    input_shape = np.shape(original_image.numpy().reshape(1, 224, 224, 3))
    pert_image = copy.deepcopy(perturbed_image)
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)

    loop_i = 0
    #x = tf.Variable(perturbed_image)
    #fs = model(preprocess_for_attack(x))
    I = I[0:num_classes]
    label = I[0]
    k_i = label

    #print(fs)  # shape=(1, 10)

    def loss_func(logits, I, k):
        # return tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
        return logits[0, I[k]]

    while k_i == label and loop_i < max_iter:
        print(k_i)
        print(label)
        #print('---------')
        pert = np.inf
        #f = (fs[0, I[k]]-fs[0, I[0]]).numpy()
        #pert = abs(f) / np.linalg.norm(tf.reshape(grad_orig, [-1]))

        for k in range(0, num_classes):
            #print(k)
            one_hot_label_k = tf.one_hot(I[k], num_classes)
            with tf.GradientTape() as tape:
                tape.watch(pert_image)
                fs = model(pert_image[..., :3])
                loss_value = loss_func(fs, I, k)
            
            cur_grad = tape.gradient(loss_value, pert_image)[..., :3]

            w_k = cur_grad
            
            #alpha_channel_k = pert_image[..., -1:]
            #w_k = w_k[...,:3]
            #w_k = w_k * tf.cast(alpha_channel_k > 0, tf.float32)

            f_k = (fs[0, I[k]]).numpy()

            pert_k = abs(f_k) / np.linalg.norm(tf.reshape(w_k, [-1]))

            if pert_k < pert:
                pert = pert_k
                w = w_k
                #alpha_channel = alpha_channel_k
        
        alpha_channel = pert_image[..., -1:]
        w = w * tf.cast(alpha_channel > 0, tf.float32)
        
        r_i = (pert + 1e-4) * w / np.linalg.norm(w)
        
        r_tot = np.float32(r_tot + r_i)
        
        pert_image = pert_image[..., :3] + (1 + overshoot) * r_tot
        pert_image = tf.concat([pert_image, alpha_channel], axis = -1)
        pert_image = tf.clip_by_value(pert_image, 0, 255)
        #pert_image = tf.Variable(pert_image)
        fs = model(pert_image.numpy().reshape(1, 224, 224, 4)[...,:3])
        k_i = np.argmax(np.array(fs).flatten())

        loop_i += 1
        
    
    mask = alpha_channel > 0
    mask_float = tf.cast(mask, tf.float32)  # Convert the boolean mask to float

    # Ensure original_image and perturbed_image are of the same type (e.g., float32)
    original_image_float = tf.cast(original_image, tf.float32)
    perturbed_image_float = tf.cast(pert_image[..., :3], tf.float32)

    # Combine the images using the mask
    result_image = original_image_float * (1 - mask_float) + perturbed_image_float * mask_float
    
    del perturbed_image, mask,original_image_float, perturbed_image_float

    return result_image 
   

def plot_images(original, temp, mask, model_name, model_type, file_name, label, model):
    width_in_inches = height_in_inches = 224 / 100
    save_paths = [f'{model_name}/{model_type}/original/', f'{model_name}/{model_type}/superpixel/', f'{model_name}/{model_type}/full/'
                 , f'{model_name}/{model_type}/deepfool_super_pixel/', f'{model_name}/{model_type}/deepfool_full/',
                 f'{model_name}/{model_type}/deepfool_negative/', f'{model_name}/{model_type}/deepfool_combine/',
                  f'{model_name}/{model_type}/neg/', f'{model_name}/{model_type}/combine/']
    
    for save_path in save_paths:
        if not os.path.exists(save_path):
            os.makedirs(save_path)

    ##original image###
    plt.figure(figsize=(width_in_inches, height_in_inches))
    plt.imshow(original)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/original/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    
    ###super pixel + attack super pixel ###
    masked_positive = np.copy(original)
    masked_positive = np.concatenate((masked_positive, np.ones((*masked_positive.shape[:-1], 1), dtype=masked_positive.dtype) * 255), axis=-1)  # Add alpha channel
    masked_positive[mask <= 0, -1] = 0
    
    plt.imshow(masked_positive)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/superpixel/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    masked_positive_adv = deepfool_attack_transparency(model, masked_positive, original, label)
    masked_positive_adv = np.squeeze(masked_positive_adv, axis=0)

    plt.imshow(masked_positive_adv)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/deepfool_super_pixel/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close() 
    
    
    ##attack negative pixel###
    masked_negative = np.copy(original)
    masked_negative = np.concatenate((masked_negative, np.ones((*masked_negative.shape[:-1], 1), dtype=masked_negative.dtype) * 255), axis=-1)  # Add alpha channel
    masked_negative[mask >= 0, -1] = 0
    
    plt.imshow(masked_negative)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/neg/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()

    masked_negative_adv = deepfool_attack_transparency(model, masked_negative, original, label)
    masked_negative_adv = np.squeeze(masked_negative_adv, axis=0)

    plt.imshow(masked_negative_adv)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/deepfool_negative/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()

    
    #attack pos+negative pixel###
    masked_combined = np.copy(original)
    masked_combined = np.concatenate((masked_combined, np.ones((*masked_combined.shape[:-1], 1), dtype=masked_negative.dtype) * 255), axis=-1)  # Add alpha channel
    masked_combined[mask == 0, -1] = 0
    
    plt.imshow(masked_combined)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/combine/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()

    masked_combined_adv = deepfool_attack_transparency(model, masked_combined, original, label)
    masked_combined_adv = np.squeeze(masked_combined_adv, axis=0)

    plt.imshow(masked_combined_adv)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/deepfool_combine/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()

    # Lime Mask를 원본 이미지에 적용하여 긍부정 시각화
    masked_negative = np.zeros_like(original)
    masked_negative[mask < 0] = [255, 0, 0]
    

    # Create a new image for positive parts (green color)
    masked_positive = np.zeros_like(original)
    masked_positive[mask > 0] = [0, 255, 0]

    # Combine the positive and negative images
    combined_image = original + masked_negative + masked_positive

    # Display the result with larger size
    plt.imshow(combined_image)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/full/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    original_image = np.expand_dims(original, axis=0)
    original_attack = deepfool(model, original_image, num_classes=2, overshoot=0.02, max_iter=50, shape=(224, 224, 3))
    original_attack = np.squeeze(original_attack, axis=0)
    plt.imshow(original_attack)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/deepfool_full/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    del masked_positive, masked_positive_adv, masked_negative, masked_negative_adv, masked_combined, masked_combined_adv
    del combined_image, original_image, original_attack
    
    plt.cla()
    plt.clf()
    
def iFGSM_and_LIME(df, i, model):
    img = cv2.imread(df['path'].iloc[i])
    selected_image = preprocess_image(img)
    y = 0 if df['label'].iloc[i]=='dogs' else 1
    Original_image = resize_and_crop(img)
    selected_image = np.expand_dims(selected_image, axis=0)
    prediction = model.predict(selected_image)
    predicted_class = np.argmax(prediction)
    explainer = lime.lime_image.LimeImageExplainer(feature_selection='auto')
    try : 
        explanation = explainer.explain_instance(selected_image[0], model.predict, top_labels=1, hide_color=0, num_samples=100)
        temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
        print('here')
        plot_images(Original_image, temp, mask, model_name, df['divide'].iloc[i], "/".join(df['path'].iloc[i].split('/')[4:]).replace("/","-"), predicted_class, model)
        del Original_image, selected_image, explainer, explanation, temp, mask, img
    except:
        pass
    
    
def file_split(range_, df, model):
    for i in tqdm(range_):
        iFGSM_and_LIME(df, i, model)

In [ ]:
os.chdir('/kaggle/working')

if __name__ == '__main__':
    original_stdout = sys.stdout
    sys.stdout = io.StringIO()
    ctypes.CDLL("libc.so.6").malloc_trim(0) 
    count = 0
    start = 500
    resize_size = 256
    crop_size = 224

    while True:
        end = 1000
        df = pd.read_csv('/kaggle/input/dog-cat-pandas/cat_dog_df.csv', index_col=0)
        df = df[df['divide']=='validation']
        model_name = 'vgg'
        model = keras.models.load_model('/kaggle/input/dog-and-cat-classifier/'+model_name+"_best_model.h5", compile=False)
        model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
        model_name = 'VGG_VALID_2'
        file_split(range(start, end), df, model)
        gc.collect()
        ctypes.CDLL("libc.so.6").malloc_trim(0)
        start += 500
        break

In [ ]:
from IPython.display import FileLink 
os.chdir('/kaggle/working/VGG_VALID_2')
archive_name = 'VGG_DEEPFOOL_LIME_VALID_2.tar.gz'
shutil.make_archive(archive_name.replace('.tar.gz', ''), 'gztar', root_dir='.', base_dir='.')
#FileLink(r'VGG_DEEPFOOL_LIME_NEW_1.tar.gz')